In [21]:
import os
os.environ['JSBSIM_ROOT_DIR'] = '/home/ubuntu/jsbsim-JSBSim-trusty-v2018a'

import sys
print ('\n'.join(sys.path))

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gym
import ray
from ray.rllib.agents.ppo import PPOAgent, DEFAULT_CONFIG
from ray.tune.logger import pretty_print

import boto3

import json

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ray/pyarrow_files
/home/ubuntu
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel
/home/ubuntu
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel

/home/ubuntu/src/cntk/bindings/python
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python36.zip
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/lib-dynload
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages
/home/ubuntu/gym-jsbsim
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/extensions
/home/ubuntu/.ipython
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ray/modin


In [2]:
# start up ray
ray.init(ignore_reinit_error=True)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-01-28_13-44-16_14767/logs.
Waiting for redis server at 127.0.0.1:25172 to respond...
Waiting for redis server at 127.0.0.1:27218 to respond...
Starting Redis shard with 10.0 GB max memory.
Starting the Plasma object store with 20.0 GB memory using /dev/shm.

View the web UI at http://localhost:8890/notebooks/ray_ui.ipynb?token=d6f0999a0e0cef724c28aadd832b2f7bbb05a1bc9a2e27db



{'node_ip_address': None,
 'redis_address': '172.31.4.39:25172',
 'object_store_address': '/tmp/ray/session_2019-01-28_13-44-16_14767/sockets/plasma_store',
 'webui_url': 'http://localhost:8890/notebooks/ray_ui.ipynb?token=d6f0999a0e0cef724c28aadd832b2f7bbb05a1bc9a2e27db',
 'raylet_socket_name': '/tmp/ray/session_2019-01-28_13-44-16_14767/sockets/raylet'}

In [3]:
config = DEFAULT_CONFIG.copy()
# num_workers is the number of actors that the agent will create. This determines the degree of parallelism that will be used.
config['num_workers'] = 20
#num_sgd_iter is the number of epochs of SGD (passes through the data) that will be used to optimize the PPO surrogate objective at each iteration of PPO
config['num_sgd_iter'] = 30
# sgd_minibatch_size is the SGD batch size that will be used to optimize the PPO surrogate objective.
config['sgd_minibatch_size'] = 512
config['model']['fcnet_hiddens'] = [100, 100]
config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed
config["num_gpus"] = 1

agent = PPOAgent(config, 'JSBSim-HeadingControlTask-A320-NoFG-v0')

2019-01-28 13:44:17,554	WARNING ppo.py:137 -- By default, observations will be normalized with MeanStdFilter
2019-01-28 13:44:17,744	INFO policy_evaluator.py:260 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
2019-01-28 13:44:20,680	INFO multi_gpu_optimizer.py:74 -- LocalMultiGPUOptimizer devices ['/gpu:0']


In [ ]:
for i in range(0):
    print("start training ", i)
    result = agent.train()
    print(pretty_print(result))

start training  0


In [4]:
#checkpoint_path = agent.save()
checkpoint_path = "/home/ubuntu/ray_results/PPO_JSBSim-HeadingControlTask-A320-NoFG-v0_2019-01-23_11-19-11n6zimocm/checkpoint_30/checkpoint-30"
print(checkpoint_path)


/home/ubuntu/ray_results/PPO_JSBSim-HeadingControlTask-A320-NoFG-v0_2019-01-23_11-19-11n6zimocm/checkpoint_30/checkpoint-30


# build json output function

In [25]:
def add_state_action_to_dict(sa_dict, sa):
    """
    add last state / action values to the dictionnary
    {pitch_rad:[],        # "attitude/pitch-rad"
     roll_rad:[],         # "attitude/roll-rad"
     sideslip_deg:[],     # "aero/beta-deg"
     v_north_fps:[],      # "velocities/v-north-fps"
     v_east_fps:[],       # "velocities/v-east-fps"
     altitude_sl_ft:[],   # "position/h-sl-ft "
     v_down_fps:[],       # "velocities/v-down-fps"
     p_radps:[],          # "velocities/p-rad_sec"
     q_radps:[],          # "velocities/q-rad_sec"
     r_radps:[],          # "velocities/r-rad_sec"
     aileron_cmd:[],      # "fcs/aileron-cmd-norm"
     elevator_cmd:[],     # "fcs/elevator-cmd-norm"
     rudder_cmd:[]}       # "fcs/rudder-cmd-norm"
    """
    if sa_dict == None:
        sa_dict = {"attitude/pitch-rad":[],
                   "attitude/roll-rad":[],
                   "aero/beta-deg":[],
                   "velocities/v-north-fps":[],
                   "velocities/v-east-fps":[],
                   "position/h-sl-ft":[],
                   "velocities/v-down-fps":[],
                   "velocities/p-rad_sec":[],
                   "velocities/q-rad_sec":[],
                   "velocities/r-rad_sec":[],
                   "fcs/aileron-cmd-norm":[],
                   "fcs/elevator-cmd-norm":[],
                   "fcs/rudder-cmd-norm":[]}
    else:
        sa_dict["attitude/pitch-rad"].append(str(sa[0]))
        sa_dict["attitude/roll-rad"].append(str(sa[1]))
        sa_dict["aero/beta-deg"].append(str(sa[2]))
        sa_dict["velocities/v-north-fps"].append(str(sa[3]))
        sa_dict["velocities/v-east-fps"].append(str(sa[4]))
        sa_dict["position/h-sl-ft"].append(str(sa[5]))
        sa_dict["velocities/v-down-fps"].append(str(sa[6]))
        sa_dict["velocities/p-rad_sec"].append(str(sa[7]))
        sa_dict["velocities/q-rad_sec"].append(str(sa[8]))
        sa_dict["velocities/r-rad_sec"].append(str(sa[9]))
        sa_dict["fcs/aileron-cmd-norm"].append(str(sa[10]))
        sa_dict["fcs/elevator-cmd-norm"].append(str(sa[11]))
        sa_dict["fcs/rudder-cmd-norm"].append(str(sa[12]))
    
    return sa_dict
                                               
def send_to_sqs(sa_dict):
    """
    send the dictionnary to the AWS queue: L2F.fifo (https://sqs.eu-west-1.amazonaws.com/190334810943/L2F.fifo)
    """
    
    # Get the service resource
    sqs = boto3.resource('sqs', region_name='eu-west-1')

    # Get the queue
    queue = sqs.get_queue_by_name(QueueName='L2F.fifo')
    
    #with open('result.json', 'w') as fp:
    #    json.dump(sa_dict, fp)

    # send json file
    response = queue.send_message(
        MessageBody=sa_dict,
        MessageGroupId='state_action_history'
    )

    # The response is NOT a resource, but gives you a message ID and MD5
    print(response.get('MessageId'))
    print(response.get('MD5OfMessageBody'))

# let's test our model

In [26]:
trained_config = config.copy()

test_agent = PPOAgent(trained_config, 'JSBSim-HeadingControlTask-A320-NoFG-v0')
test_agent.restore(checkpoint_path)

2019-01-28 14:17:56,923	WARNING ppo.py:137 -- By default, observations will be normalized with MeanStdFilter
2019-01-28 14:17:56,926	INFO policy_evaluator.py:260 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
2019-01-28 14:17:58,749	INFO multi_gpu_optimizer.py:74 -- LocalMultiGPUOptimizer devices ['/gpu:0']


In [27]:
env = gym.make('JSBSim-HeadingControlTask-A320-NoFG-v0')
obs_space = env.observation_space
act_space = env.action_space
print(obs_space, act_space)
state = env.reset()
done = False
cumulative_reward = 0
i = 0
act_stat_reward = []
sa_dict = None
while i != 50:
    #print(gym.tasks.last_state())
    action = test_agent.compute_action(state)
    state, reward, done, _ = env.step(action)
    print(action, state, reward)
    mergedlist = []
    mergedlist.extend(state)
    mergedlist.extend(action)
    state_action_list = mergedlist
    sa_dict = add_state_action_to_dict(sa_dict, state_action_list)
    
    mergedlist.extend([reward])
    
    act_stat_reward.append(mergedlist)
    cumulative_reward += reward
    i += 1

print("SA_DICT", sa_dict)

send_to_sqs(json.dumps(sa_dict))
print(cumulative_reward)
print("aileron_cmd,elevator_cmd,rudder_cmd,pitch_rad,roll_rad,sideslip_deg,v_north_fps,v_east_fps,altitude_sl_ft,v_down_fps,p_radps,q_radps,r_radps,reward")
for e in act_stat_reward:
        print(",".join(str(v) for v in e))
with open('result.json', 'w') as fp:
    json.dump(sa_dict, fp)

Box(10,) Box(3,)
[-0.7750001   0.93363976  0.95278794] [-5.51263190e-02 -8.15749734e-02  2.89826036e+00 -1.22530006e+00
 -8.26485314e+02  5.00063417e+03 -4.37296360e+00 -9.15218469e-01
 -5.78656438e-01 -3.85260468e-01] -2.038766636117786e-33
[-1.2160997  -0.06992811  0.8150565 ] [-1.43910183e-01 -3.46298282e-01  7.05719367e+00 -4.84780237e+00
 -8.26085080e+02  5.00010709e+03  1.11529846e+01 -1.78634514e+00
 -1.08153799e-01 -1.52248670e-01] -0.050881652220987054
[ 0.48597392 -0.04473329  0.54341406] [-1.32076941e-01 -6.07805233e-01  7.27103166e+00 -1.02075839e+01
 -8.25248697e+02  4.99629119e+03  2.52763050e+01 -7.14575045e-01
  1.95876239e-01  1.11354528e-01] -0.05729633405868968
[-0.44558635  0.6602934   1.392009  ] [-1.31593848e-01 -7.68801146e-01  6.90596328e+00 -2.19732816e+01
 -8.23802962e+02  4.99084229e+03  2.88693712e+01 -9.86080261e-01
 -2.31646971e-01 -1.26299142e-01] -0.0343106399681083
[-1.1558917   0.18189526  0.69784164] [-1.71789903e-01 -1.05110431e+00  7.14212954e+00 -3

[ 0.27803135 -1.7822566  -2.5590575 ] [-1.04837311e+00 -1.44557492e+00 -1.32033481e+00 -2.69796831e+02
 -3.20505447e+02  1.72264297e+03  7.35467640e+02  7.35719956e-01
  4.88829925e-01  3.76346262e-01] -0.9917491862368045
[ 0.17712069  1.417923   -0.07041129] [-1.01661912e+00 -1.22978622e+00 -2.83928329e+00 -2.75566406e+02
 -3.20735178e+02  1.57522082e+03  7.38039657e+02  5.87401874e-01
 -1.63063829e-01 -9.10858830e-02] -0.9942065115926605
[-0.4896727   0.53957665  2.8536434 ] [-1.11147300e+00 -1.25767756e+00  1.47255902e+00 -2.75792936e+02
 -3.21730197e+02  1.42720315e+03  7.43265809e+02 -3.51272487e-01
 -3.93414131e-01 -5.08599441e-01] -1.0070172074948318
[ 2.7190623  -4.614518   -0.81773084] [-1.10094117e+00 -1.19866627e+00  7.94484018e-01 -2.61722944e+02
 -3.22829069e+02  1.27755479e+03  7.52995416e+02  6.96195541e-01
  7.45500192e-01  4.46737476e-01] -1.0477069437465936
[-1.1675156 -1.9320683 -0.437608 ] [-9.44676619e-01 -9.60381532e-01 -2.93583054e+00 -2.79265642e+02
 -3.18823989

de53e3bb-2a4e-4339-b426-98370e2693f1
30af203826f8ec20b04a294b06f11fa9
-23.961544402936966
aileron_cmd,elevator_cmd,rudder_cmd,pitch_rad,roll_rad,sideslip_deg,v_north_fps,v_east_fps,altitude_sl_ft,v_down_fps,p_radps,q_radps,r_radps,reward
-0.05512631901782417,-0.08157497340037886,2.898260364067641,-1.2253000583555715,-826.4853138608341,5000.63417076692,-4.372963604263491,-0.9152184688503663,-0.5786564384324328,-0.38526046765135674,-0.7750001,0.93363976,0.95278794,-2.038766636117786e-33
-0.14391018257502844,-0.3462982818898856,7.057193666247725,-4.84780237485968,-826.085079502216,5000.107094358653,11.15298463961725,-1.7863451428880757,-0.10815379877552608,-0.1522486704221423,-1.2160997,-0.06992811,0.8150565,-0.050881652220987054
-0.13207694128689582,-0.6078052329104497,7.2710316649781435,-10.207583917689954,-825.2486966017257,4996.291188210249,25.27630496192293,-0.7145750452770078,0.19587623876231983,0.11135452774133135,0.48597392,-0.044733286,0.54341406,-0.05729633405868968
-0.131593847